In [3]:
import pandas as pd
import itertools
import numpy as np
import openpyxl

### SETS

In [6]:

# Velg antall av de ulike settene her
set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}
sets_excel_output_path = 'SetData-sheets.xlsx'

t_values = range(0, 5)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 4)
f_values = range(1, 2)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 3)  # Assuming you want values from 1 to 3122 for 'r'
i_values = range(1, 4) # 18 installations
s_values = range(1, 3) # 11 systems
w_values = range(1, 4) # 100 scenarios
a_values = range(0, 6)
s_values = range(1, 6)
a_values = range(1,4)

for t in t2_values:
    print(t)



# set_sheets = {'Power systems': s_values, 'Ages': a_values, 'Routes': r_values, 'Time periods':t_values, 'Time periods 1': t1_values, 'Time periods 2': t2_values, 'Fuel types': f_values, 'Installations': n_values, 'Scenarios': w_values}
columnsname = ['S', 'A', 'R', 'T', 'T1', 'T2', 'F', 'N', 'O']




# with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
#     df.to_excel(writer, sheet_name=sheet, index=False)


2
3


#### Generere kombinasjoner av indekser til parameterne

In [24]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)

#### Inputdata

In [5]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.29, 2:0.30, 3:0.82, 4:0.73, 5:0.87, 6:0.69}

# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000181667, 2: 0.0001525, 3: 0.000109444, 4: 0.00004775, 5: 0.0000722222, 6: 0.00000791667}
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0}


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {1: 0, 2: 0.50, 3: 0.55, 4: 0.70, 5: 0.80, 6: 0.95} # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

initial_fleet = 1

#### Lese inn data fra rutegenereringsprogrammet

In [14]:
# # Leser inn nødvendig MJ på å kjøre hver rute
# mj_route = pd.read_csv('mj_route.csv')
# mj_route_el = pd.read_csv('mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
# hours_route = pd.read_csv('../route_generator/generated_datafiles/hours_route.csv')
# hours_route_el = pd.read_csv('../route_generator/generated_datafiles/hours_route_strøm.csv')



# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = 'Parameterdata-sheets.xlsx'

#### Initial fleet

In [29]:
# Create a DataFrame from the combinations
df_initialfleet = pd.DataFrame(combinations_s_a, columns=['S', 'A'])
#df_initialfleet['Value'] = [0]


# Set chosen values in 'Value' column
# df_initialfleet.loc[(df_initialfleet['S'] == 1) & (df_initialfleet['A'] == 3), 'Value'] = 2
# df_initialfleet.loc[(df_initialfleet['S'] == 2) & (df_initialfleet['A'] == 4), 'Value'] = 1


# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_initialfleet.to_excel(writer, sheet_name='Initial fleet', index=False)

df_initialfleet

,S,A


##### Cost of fuel per route

In [14]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route.to_dict()
distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

specific_distance_dict = mj_route_el.to_dict()
specific_distance_dict= specific_distance_dict['mj_route']
electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(distance_dict)
mj_route

print(electric_distance_dict)

{0: 134260.0, 1: 155865.2, 2: 104049.6, 3: 168674.8, 4: 165154.8, 5: 183922.0, 6: 131370.4, 7: 157610.4, 8: 142274.8, 9: 120660.0, 10: 125160.0, 11: 111050.4, 12: 128500.0, 13: 211762.0, 14: 91540.4, 15: 84670.0, 16: 76814.8, 17: 112020.0, 18: 215165.2, 19: 197749.6, 20: 238694.8, 21: 241654.8, 22: 226582.0, 23: 174990.4, 24: 169870.4, 25: 204934.8, 26: 146840.0, 27: 144860.0, 28: 146350.4, 29: 139720.0, 30: 246422.0, 31: 146360.4, 32: 148690.0, 33: 136434.8, 34: 150840.0, 35: 192314.8, 36: 208300.0, 37: 213100.0, 38: 199467.2, 39: 158367.2, 40: 211087.2, 41: 162700.0, 42: 170285.2, 43: 203265.2, 44: 164607.2, 45: 207485.2, 46: 215307.2, 47: 167657.2, 48: 163335.2, 49: 164360.0, 50: 162525.2, 51: 199912.8, 52: 195592.8, 53: 144060.0, 54: 207660.0, 55: 179592.8, 56: 151429.6, 57: 184089.6, 58: 140940.0, 59: 189589.6, 60: 129830.0, 61: 120319.6, 62: 125732.8, 63: 138469.6, 64: 203669.6, 65: 211865.2, 66: 174925.2, 67: 235485.2, 68: 206309.6, 69: 216614.8, 70: 226074.8, 71: 209485.2, 72: 

Generere alle kombinasjoner av R og F

In [15]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [19]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
def calculate_value(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

df_fuelprice

NameError: name 'df_fuelprice' is not defined

##### Skriver Cost of fuel til excel

In [17]:


# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [21]:

# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

Empty DataFrame
Columns: [F, R]
Index: []


In [19]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
def calculate_emissions(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ_ttw[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)

# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)

df_ems

,F,R,Value
0,1,1,20.511974
1,1,2,23.812774
2,1,3,15.896490
3,1,4,25.769799
4,1,5,25.232020
...,...,...,...
18721,6,3117,0.000000
18722,6,3118,0.000000
18723,6,3119,0.000000
18724,6,3120,0.000000


##### MAX EMISSIONS PER TIME PERIOD

In [20]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
print(df_max_ems)

   T
0  1
1  2
2  3
3  4
4  5
5  6


In [21]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)

# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,1,165000.0
1,2,82500.0
2,3,74250.0
3,4,49500.0
4,5,33000.0
5,6,8250.0


##### TIME USED

In [22]:
# LAGE TIME USED

# Create T_rs
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route.to_dict()
route_dict = route_dict['Hours']
hours_route_dict = {key + 1: value for key, value in route_dict.items()}

# lager egne distanser for elektrisk
hours_route_el_dict = hours_route_el.to_dict()
hours_route_el_dict= hours_route_el_dict['Hours']
hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}

print(hours_route_dict)
print(hours_route_el_dict)


# funksjon for å fylle inn verdier i Trs
def calculate_timeused(row):
    value = hours_route_el_dict[row['R']] if row['S'] == 11 else hours_route_dict[row['R']]
    return value

# Legger til verdiene i Value-kolonne i dataframen
df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


   Hours
0   24.0
1   24.0
2   24.0
3   24.0
4   24.0
   Hours
0   24.0
1   48.0
2   24.0
3   48.0
4   48.0
{1: 24.0, 2: 24.0, 3: 24.0, 4: 24.0, 5: 24.0, 6: 48.0, 7: 24.0, 8: 24.0, 9: 24.0, 10: 24.0, 11: 24.0, 12: 24.0, 13: 24.0, 14: 48.0, 15: 24.0, 16: 24.0, 17: 24.0, 18: 24.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 24.0, 25: 24.0, 26: 48.0, 27: 24.0, 28: 24.0, 29: 24.0, 30: 24.0, 31: 48.0, 32: 24.0, 33: 24.0, 34: 24.0, 35: 24.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 24.0, 41: 48.0, 42: 24.0, 43: 24.0, 44: 48.0, 45: 24.0, 46: 48.0, 47: 48.0, 48: 24.0, 49: 24.0, 50: 24.0, 51: 24.0, 52: 48.0, 53: 48.0, 54: 24.0, 55: 48.0, 56: 48.0, 57: 24.0, 58: 48.0, 59: 24.0, 60: 48.0, 61: 24.0, 62: 24.0, 63: 24.0, 64: 24.0, 65: 48.0, 66: 48.0, 67: 24.0, 68: 48.0, 69: 48.0, 70: 48.0, 71: 48.0, 72: 48.0, 73: 48.0, 74: 48.0, 75: 48.0, 76: 48.0, 77: 48.0, 78: 48.0, 79: 48.0, 80: 48.0, 81: 48.0, 82: 48.0, 83: 48.0, 84: 48.0, 85: 48.0, 86: 48.0, 87: 48.0, 88: 48.0, 89: 48.0, 90: 48.0, 